In [1]:
!pip install shap



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
df=pd.read_csv(r"C:/Users/HP/PCOD-DETECTION/data/processed/Day5-preprocessed.csv")
df.head()

,Follicle No. (R),Follicle No. (L),Skin darkening (Y/N),hair growth(Y/N),Weight gain(Y/N),Cycle(R/I),Fast food (Y/N),Pimples(Y/N),AMH(ng/mL),Weight (Kg),BMI,Hair loss(Y/N),Waist(inch),Hip(inch),Avg. F size (L) (mm),PCOS (Y/N)
0,-0.821501,-0.741039,-0.666221,-0.614452,-0.779194,-0.622331,0.967204,-0.978019,-0.604849,-1.364256,-1.234925,-0.911322,-1.068363,-0.503842,0.837271,0
1,-0.370677,-0.741039,-0.666221,-0.614452,-0.779194,-0.622331,-1.033908,-0.978019,-0.696746,0.485719,0.146043,-0.911322,-0.512299,0.000467,-0.004051,0
2,1.883440,1.623978,-0.666221,-0.614452,-0.779194,-0.622331,0.967204,1.022475,0.171168,0.830323,0.244683,1.097307,0.599828,0.504776,0.837271,1
3,-1.046912,-0.977540,-0.666221,-0.614452,-0.779194,-0.622331,-1.033908,-0.978019,-0.749502,0.485719,1.329730,-0.911322,0.599828,1.009084,-0.004051,0
4,-0.596089,-0.741039,-0.666221,-0.614452,-0.779194,-0.622331,-1.033908,-0.978019,-0.572515,-0.693187,-1.037644,1.097307,-1.068363,-0.251687,0.276390,0


In [19]:
# ✅ Compatibility Fix
import numpy as np
np.bool = bool  # Fix for SHAP compatibility with NumPy ≥1.24

# 📦 Libraries
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import os

# 📌 Load Preprocessed Data (assuming df is already defined)
X = df.drop("PCOS (Y/N)", axis=1)
y = df["PCOS (Y/N)"]

# 📊 Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🤖 Train Random Forest Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 🔍 SHAP Explainer
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test, check_additivity=False)  # Disable strict additivity check

# 📁 Create folder to save plots
os.makedirs("shap_outputs", exist_ok=True)

# 📈 SHAP Summary Plot (Global Feature Importance)
plt.figure()
shap.summary_plot(shap_values, show=False)
plt.savefig("shap_outputs/shap_summary_plot.png", bbox_inches='tight')
plt.close()

# 📉 SHAP Waterfall Plot (Local Explanation for Sample 0 and Class 1)
sample_index = 0  # You can change this to view other samples

shap_single = shap.Explanation(
    values=shap_values.values[sample_index, 1],
    base_values=shap_values.base_values[sample_index, 1],  # ✅ Final Fix: sample-wise base value
    data=shap_values.data[sample_index],
    feature_names=X_test.columns
)

plt.figure()
shap.plots.waterfall(shap_single, show=False)
plt.savefig("shap_outputs/shap_waterfall_plot.png", bbox_inches='tight')
plt.close()

print("✅ SHAP summary and waterfall plots saved successfully in 'shap_outputs/' folder.")


✅ SHAP summary and waterfall plots saved successfully in 'shap_outputs/' folder.
